In [1]:
# Importing pandas and reading eBay comments data into a DataFrame
import pandas as pd

csv_file_path = "Data/ebay_comments.csv"

df = pd.read_csv(csv_file_path)

df


,Search,Product Title,Condition,Price,Seller,Review,Review - Title,Review - Star,Review - Author,Review - Date,Review - Verified Purchase
0,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,musicmagpie,£634.99,NaN,Probably as I had hoped for as the phone is in...,Great but heavy.,4 stars,stickyprobs,"28 Jun, 2023",Yes
1,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,new,£634.99,ssa2234ssa,"Fantastic,new and sealed as described. It come...",Fantastic,5 stars,helwa1078-voxze...,"29 Sep, 2022",Yes
2,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,pre-owned,£634.99,phones2gadgets,It’s a good phone.. Had a new 13 model but fo...,Will do for me for a few years.. hopefully..,5 stars,paulprior249,"20 Jun, 2023",Yes
3,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,phonesdirect_com,£634.99,NaN,Perfect quality as if its new and it has 100% ...,Goood,5 stars,88s275,"16 Aug, 2023",Yes
4,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,new,£634.99,sell-phones,I love it fantastic phone,Perfect 👍,5 stars,sarolga,"04 Jun, 2022",Yes
...,...,...,...,...,...,...,...,...,...,...,...
64009,Macbook,"Apple MacBook Air 13.3"" 2019 Silver (256 GB SS...",new,£175.00,dgeltd,great device,great device,5 stars,spils-natha,"18 Sep, 2019",Yes
64010,Macbook,"Apple MacBook Air 13.3"" 2019 Silver (256 GB SS...",new,£215.00,stefidel,Great quality,I love Apple products,5 stars,kamiliosk,"11 Dec, 2019",Yes
64011,Macbook,"Apple MacBook Air 13.3"" 2019 Silver (256 GB SS...",new,£215.00,authenticbeanies,Product exactly as described at a great price....,Exactly as described,5 stars,matcham-65,"17 Sep, 2019",Yes
64012,Macbook,"Apple MacBook Air 13.3"" 2019 Silver (256 GB SS...",NaN,£215.00,NaN,Latest Technology With Amazing Price,Amazing Technology,5 stars,macansysltd,"19 Sep, 2019",No


In [25]:
# Convert DataFrame 'df' to a list of dictionaries, each representing a row
results = df.to_dict(orient='records')
results


[{'Search': 'Iphone',
  'Product Title': 'Apple iPhone 13 Pro Max - 128GB - Alpine Green - Unlocked - Very Good Condition',
  'Condition': 'musicmagpie',
  'Price': '£634.99',
  'Seller': nan,
  'Review': 'Probably as I had hoped for as the phone is in good condition happy with that and the performance is excellent it’s only down side is the 13 pro max is on the heavy side making it difficult to slip in a pocket and be comfortable walking. Heavy is the price you pay for a large screen and amazing camera performance etc.',
  'Review - Title': 'Great but heavy.',
  'Review - Star': '4 stars',
  'Review - Author': 'stickyprobs',
  'Review - Date': '28 Jun, 2023',
  'Review - Verified Purchase': 'Yes'},
 {'Search': 'Iphone',
  'Product Title': 'Apple iPhone 13 Pro Max - 128GB - Alpine Green - Unlocked - Very Good Condition',
  'Condition': 'new',
  'Price': '£634.99',
  'Seller': 'ssa2234ssa',
  'Review': 'Fantastic,new and sealed as described. It comes with screen and camera protector, ca

In [18]:
# This script scrapes eBay for MacBook listings, extracting and filtering product URLs.

import requests
from bs4 import BeautifulSoup
import time

macbook_url = "https://www.ebay.co.uk/b/bn_450926?_pgn="

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0'
}

for i in range(167):
    print(i)
    try:
        search_url = macbook_url + str(i+1)
        response = requests.get(search_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        temp = [a["href"] for a in soup.find_all("a", class_="s-item__link")]
        urls += temp
    except:
        break
    time.sleep(2)

urls = [link for link in urls if "ebay.co.uk/itm/" in link]
urls = list(set(urls))
print(urls)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166


['https://www.ebay.co.uk/itm/334605624265?hash=item4de80ca3c9:g:IZQAAOSw91tjWJe6',
 'https://www.ebay.co.uk/itm/386306095960?hash=item59f1a33b58:g:i2YAAOSwO9pkaer2',
 'https://www.ebay.co.uk/itm/305015653849?hash=item47045985d9:g:bVUAAOSwzm1kqUQk',
 'https://www.ebay.co.uk/itm/256315261026?hash=item3bad948062:g:1qEAAOSwwhRlYdy1',
 'https://www.ebay.co.uk/itm/276190993542?hash=item404e441c86:g:0UEAAOSwHHVlSp5H',
 'https://www.ebay.co.uk/itm/175853296600?hash=item28f1ac5bd8:g:vugAAOSwn71k00wu&var=475262999123',
 'https://www.ebay.co.uk/itm/126162629509?epid=20032113102&hash=item1d5fe0f385:g:gxsAAOSwdS9lQBSf&amdata=enc%3AAQAIAAAA0EVyyouNa3mNGCw2%2BCUJ6SB3FvzQY1D6hb79SeNpSO2tjCKvAOzy3JWMJNVrNAJdYzxnAg8hKjdVbxeSZlFNAE7BFvDEbNdSiU9arZbs2iRIJI9kAAjJ3huSrwlS3%2FSrajBd5ovqkGLtAuC80MlIeB1TRVWT4Q%2FOjX2GuSztxDV989KwSmK%2F2z9gx6B6%2Bc%2BxcG1W6MGQQwo6R%2BTBAtuiymHgBY0k2JjYbEPfYRZ56KDTtGRROLUntE%2F1soK1nYJL9um5grymCWvzwhrWURXQmdQ%3D%7Ctkp%3ABFBM-u7ZxYNj',
 'https://www.ebay.co.uk/itm/115966709117?ha

In [37]:
mac_urls = urls
mac_urls

['https://www.ebay.co.uk/itm/334605624265?hash=item4de80ca3c9:g:IZQAAOSw91tjWJe6',
 'https://www.ebay.co.uk/itm/386306095960?hash=item59f1a33b58:g:i2YAAOSwO9pkaer2',
 'https://www.ebay.co.uk/itm/305015653849?hash=item47045985d9:g:bVUAAOSwzm1kqUQk',
 'https://www.ebay.co.uk/itm/256315261026?hash=item3bad948062:g:1qEAAOSwwhRlYdy1',
 'https://www.ebay.co.uk/itm/276190993542?hash=item404e441c86:g:0UEAAOSwHHVlSp5H',
 'https://www.ebay.co.uk/itm/175853296600?hash=item28f1ac5bd8:g:vugAAOSwn71k00wu&var=475262999123',
 'https://www.ebay.co.uk/itm/126162629509?epid=20032113102&hash=item1d5fe0f385:g:gxsAAOSwdS9lQBSf&amdata=enc%3AAQAIAAAA0EVyyouNa3mNGCw2%2BCUJ6SB3FvzQY1D6hb79SeNpSO2tjCKvAOzy3JWMJNVrNAJdYzxnAg8hKjdVbxeSZlFNAE7BFvDEbNdSiU9arZbs2iRIJI9kAAjJ3huSrwlS3%2FSrajBd5ovqkGLtAuC80MlIeB1TRVWT4Q%2FOjX2GuSztxDV989KwSmK%2F2z9gx6B6%2Bc%2BxcG1W6MGQQwo6R%2BTBAtuiymHgBY0k2JjYbEPfYRZ56KDTtGRROLUntE%2F1soK1nYJL9um5grymCWvzwhrWURXQmdQ%3D%7Ctkp%3ABFBM-u7ZxYNj',
 'https://www.ebay.co.uk/itm/115966709117?ha

In [34]:
# Define the user-agent header to mimic a web browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

# Loop through the list of URLs for MacBook products
for url in range(len(mac_urls)):
    print(url)
    
    # Send a GET request to the URL with the defined headers
    response = requests.get(mac_urls[url], headers=headers)
    
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the elements on the page
    try:
        product_title_element = soup.find('span', {'class': 'ux-textspans ux-textspans--BOLD'})
    except:
        continue
    
    # Extract information from the found elements
    try:
        product_title = product_title_element.get_text(strip=True)
    except:
        continue
    
    price_element = soup.find('div', {'data-testid': 'x-price-primary'})
    price = price_element.find(class_="ux-textspans").text.strip()
    
    reviews_element = soup.find('div', {'class': 'x-review-details__allreviews'})
    reviews_link = reviews_element.find('a', {'data-testid': 'ux-action'})['href']
    reviews_count = int(reviews_element.find('a', {'data-testid': 'ux-action'}).text.strip().split()[2])
    
    # Calculate the number of pages for reviews
    if reviews_count % 10 == 0:
        pages_count = int(reviews_count/10)
    else:
        pages_count = int(reviews_count/10)+1
    
    # Initialize an empty list to store review details
    results = list()
    
    # Loop through each page of reviews
    for i in range(pages_count):
        # Construct the URL for the current review page
        page = reviews_link + "&pgn=" + str(i+1)
        response = requests.get(page, headers=headers)
        
        # Extract HTML content from the response
        html_content = response.content

        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all the review elements on the page
        review_elements = soup.find_all(class_="ebay-review-section")

        # Loop through the review elements and extract relevant information
        for review_element in review_elements:
            try:
                review_text = review_element.find(class_="review-item-content").get_text(strip=True, separator='\n')
            except:
                review_text = ""
                pass
            try:
                review_title = review_element.find(class_="review-item-title").get_text(strip=True, separator='\n')
            except:
                review_title = ""
                pass
            try:
                review_star = review_element.find(class_="star-rating")['aria-label']
            except:
                review_star = ""
            try:
                review_author = review_element.find(class_="review-item-author").get_text(strip=True, separator='\n')
            except:
                review_author = ""
                pass
            try:
                review_date = review_element.find(class_="review-item-date").get_text(strip=True, separator='\n')
            except:
                review_date = ""
                pass
            try:
                review_verifiedPurchase = review_element.find_all(class_="rvw-val")[0].get_text(strip=True, separator='\n')
            except:
                review_verifiedPurchase = ""
                pass
            try:
                condition = review_element.find_all(class_="rvw-val")[1].get_text(strip=True, separator='\n')
            except:
                condition = ""
                pass
            try:
                seller = review_element.find_all(class_="rvw-val")[2].get_text(strip=True, separator='\n')
            except:
                seller = ""
                pass
            
            # Append the extracted information to the 'results' list
            results.append({'Search': 'Macbook', 'Product Title': product_title, 'Condition': condition, 'Price': price, 'Seller': seller, 
                            'Review': review_text, 'Review - Title': review_title, 'Review - Star': review_star,
                            'Review - Author': review_author, 'Review - Date': review_date,  
                            'Review - Verified Purchase': review_verifiedPurchase})
            
            # Create a DataFrame from the 'results' list
            df = pd.DataFrame(results)
            
            # Save the DataFrame to a CSV file
            df.to_csv('ebay_comments.csv', index=False)
        
        # Pause for 2 seconds before making the next request (to avoid being blocked)
        time.sleep(2)

# Display the final DataFrame
df


296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545


KeyboardInterrupt: 

In [35]:
#df = pd.DataFrame(results)
df = df.drop_duplicates()
df.to_csv('ebay_comments.csv', index=False)
df

,Search,Product Title,Condition,Price,Seller,Review,Review - Title,Review - Star,Review - Author,Review - Date,Review - Verified Purchase
0,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,musicmagpie,£634.99,NaN,Probably as I had hoped for as the phone is in...,Great but heavy.,4 stars,stickyprobs,"28 Jun, 2023",Yes
1,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,new,£634.99,ssa2234ssa,"Fantastic,new and sealed as described. It come...",Fantastic,5 stars,helwa1078-voxze...,"29 Sep, 2022",Yes
2,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,pre-owned,£634.99,phones2gadgets,It’s a good phone.. Had a new 13 model but fo...,Will do for me for a few years.. hopefully..,5 stars,paulprior249,"20 Jun, 2023",Yes
3,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,phonesdirect_com,£634.99,NaN,Perfect quality as if its new and it has 100% ...,Goood,5 stars,88s275,"16 Aug, 2023",Yes
4,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,new,£634.99,sell-phones,I love it fantastic phone,Perfect 👍,5 stars,sarolga,"04 Jun, 2022",Yes
...,...,...,...,...,...,...,...,...,...,...,...
65308,Macbook,"Apple MacBook Air 13in (256GB SSD, M1, 8GB) La...",pre-owned,US $799.00,quickshipelectr...,This is a exceptionally good looking computer....,Excelllent,5 stars,elfig63,"17 Aug, 2021",Yes
65309,Macbook,"Apple MacBook Air 13in (256GB SSD, M1, 8GB) La...",pre-owned,US $799.00,jy161069,Absolutely amazing! Battery life is so darn go...,M1 MacBook Air,5 stars,shgre_44,"26 Jun, 2022",Yes
65310,Macbook,"Apple MacBook Air 13in (256GB SSD, M1, 8GB) La...",pre-owned,US $799.00,quickshipelectr...,This computer blows every other laptop I’ve ow...,Incredible Computer,5 stars,cbyardsale,"14 Sep, 2021",Yes
65311,Macbook,"Apple MacBook Air 13in (256GB SSD, M1, 8GB) La...",pre-owned,US $799.00,ricardvasque-2,I am happy with this MacBook. It wasn’t new bu...,MG,5 stars,mglo8384,"28 Sep, 2021",Yes


In [36]:
# Display the counts of unique counts of product types
df.Search.value_counts()


Iphone         51903
AirPods         5231
Macbook         3104
Apple Watch     2598
iPad            2354
Name: Search, dtype: int64

In [43]:
# Drop duplicate rows from the DataFrame 'df' in place
df.drop_duplicates(inplace=True)

# Save the updated DataFrame to a CSV file named 'ebay_comments.csv'
df.to_csv('ebay_comments.csv', index=False)

# Display the updated DataFrame
df


,Search,Product Title,Condition,Price,Seller,Review,Review - Title,Review - Star,Review - Author,Review - Date,Review - Verified Purchase
0,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,musicmagpie,£634.99,NaN,Probably as I had hoped for as the phone is in...,Great but heavy.,4 stars,stickyprobs,"28 Jun, 2023",Yes
1,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,new,£634.99,ssa2234ssa,"Fantastic,new and sealed as described. It come...",Fantastic,5 stars,helwa1078-voxze...,"29 Sep, 2022",Yes
2,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,pre-owned,£634.99,phones2gadgets,It’s a good phone.. Had a new 13 model but fo...,Will do for me for a few years.. hopefully..,5 stars,paulprior249,"20 Jun, 2023",Yes
3,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,phonesdirect_com,£634.99,NaN,Perfect quality as if its new and it has 100% ...,Goood,5 stars,88s275,"16 Aug, 2023",Yes
4,Iphone,Apple iPhone 13 Pro Max - 128GB - Alpine Green...,new,£634.99,sell-phones,I love it fantastic phone,Perfect 👍,5 stars,sarolga,"04 Jun, 2022",Yes
...,...,...,...,...,...,...,...,...,...,...,...
65951,Apple Watch,Apple MTF32LLA Watch Series 3 GPS 42 Mm Space ...,pre-owned,US $219.99,vipoutlet,Watch works great and it’s in good condition w...,Works great,5 stars,jenkfarrell09,"24 May, 2019",Yes
65952,Apple Watch,Apple MTF32LLA Watch Series 3 GPS 42 Mm Space ...,pre-owned,US $219.99,vipoutlet,Condition is very very good! 170$ for series 3...,Very goon,5 stars,feshenclub85,"03 Jun, 2019",Yes
65953,Apple Watch,Apple MTF32LLA Watch Series 3 GPS 42 Mm Space ...,refurbished,US $219.99,blinq,Came just as it was described. Looks great!,great quality for the money,5 stars,acc2584,"15 May, 2019",Yes
65954,Apple Watch,Apple MTF32LLA Watch Series 3 GPS 42 Mm Space ...,pre-owned,US $219.99,phone_cellar,My son loves it!,Apple watch series 3,4 stars,blacconfederate,"27 Dec, 2019",Yes


In [44]:
# Display the counts of unique counts of product types
df.Search.value_counts()


Iphone         51903
AirPods         5231
Apple Watch     3229
Macbook         3104
iPad            2354
Name: Search, dtype: int64